In [1]:
#import libraries here
# Base
import numpy as np
import PIL
import os
from datetime import datetime
import pandas as pd


# Scikit-learn

# Display
from IPython.display import Image

# Torch
import torch
from torchvision import transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader, random_split, Dataset

# Other utility functions
from accelerate import Accelerator

c:\Users\wwwia\Documents\Projects\htr-scanner\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#!unzip -qq data/archive.zip -d dataset/

In [ ]:
Image(filename=os.path.expandvars('dataset\data\data\input_1_1_12.jpg'))

In [ ]:
# GAS GAS GAS 
accelerator = Accelerator(mixed_precision="fp16")
# Check the device
device = accelerator.device

print(f"The default device is set to {torch.cuda.get_device_name(device)}")

In [ ]:
#data exploration
data = pd.read_csv('dataset\chinese_mnist.csv', delimiter=',')
data.dataframeName = 'chinese_mnist_df.csv'

data.loc[(data['suite_id'] == 100) & (data['sample_id'] == 10) & (data['code'] == 10)]

In [2]:
# Create custom image dataset (so the images are labeled):
class HRDigitDataSet(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, target_transform=None, seed=69):
        df = pd.read_csv(csv_file)
        df = df.sample(n = len(df), random_state=seed)
        self.labels_df = df.reset_index(drop=True)
        self.img_dir = img_dir        
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return self.labels_df.shape[0]
    
    def __getitem__(self, index):
        # get image file path using information in the csv file
        image_name = os.path.join(self.img_dir, 
        f"input_{self.labels_df.iloc[index, 0]}_{self.labels_df.iloc[index, 1]}_{self.labels_df.iloc[index, 2]}.jpg")
        # get image tensor using the image file path
        image = read_image(image_name)  # read the image as a grayscale matrix 
        # get image label from the csv file
        label = self.labels_df.iloc[index, 3]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        image = torch.as_tensor(image, dtype=torch.float32) # convert matrix into Pytorch tensor
        label = torch.as_tensor(label, dtype=torch.int64) # convert label into Pytorch tensor
        return image, label

In [3]:
#Test to see if dataset object works
dataset = HRDigitDataSet(csv_file='dataset\chinese_mnist.csv',img_dir='dataset\data\data')
dataset.__getitem__(2)

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 tensor(0))

In [ ]:
# Split dataset into training, validation and testing portions
# 60% train / 25% val / 25% test 
def loadDataset(dataset, batch_size=32, train_split = 0.6, val_split = 0.25, test_split = 0.25, random_seed=69):
    # ensure splits add up to 1:
    assert train_split + val_split + test_split == 1.0, "Splits must add up to 1.0"
    # Determine the split sizes
    total_size = len(dataset)
    train_size = int(total_size * train_split)
    val_size = int(total_size * val_split)
    test_size = int(total_size * test_split)
    # This ensures reproducibility
    torch.manual_seed(random_seed)
    # Split the dataset and create loaders
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle= True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle= False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader 

In [ ]:
#CRNN model:
import torch.nn as nn
#1. CNN Feature Extractor
class CNNBlock(nn.Module):
    def __init__(self, input_channel, output_channel, kernel_sizes, strides, paddings, batch_norm: bool = False):
        super(CNNBlock, self).__init__()
        self.do_batch_norm = batch_norm
        self.conv = nn.Conv2d(input_channel, output_channel, kernel_sizes, strides, paddings)
        self.bn = nn.BatchNorm2d(output_channel)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        if self.do_batch_norm:
            x = self.bn(x)
        x = self.relu(x)
        return x


class CRNN1(nn.Module):
    def __init__(self, img_channel, img_dim, num_class, collapse_layer_hidden=64, rnn_hidden=256):
        super(CRNN1, self).__init__()
        # CNN block
        self.cnn = nn.Sequential(
            CNNBlock(img_channel, 16, 3, 1, 1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            CNNBlock(16, 32, 3, 1, 1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            CNNBlock(32, 64, 3, 1, 1),
            CNNBlock(64, 128, 3, 1, 1),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(0.2),
            CNNBlock(128, 256, 3, 1, 1, batch_norm=True),
            nn.Dropout(0.2),
            CNNBlock(256, 512, 3, 1, 1, batch_norm=True),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(0.2),
            CNNBlock(512, 512, 2, 1, 0, batch_norm=True),
        )
        output_height = img_dim // 16 - 1
        self.collapse_features = nn.Linear(512 * output_height, collapse_layer_hidden)
    def forward(self, images):
        # Extract features
        conv = self.cnn(images)
        # Reformat array
        batch, channel, height, width = conv.size()
        conv = conv.view(batch, channel * height, width)
        conv = conv.permute(2, 0, 1)
        conv = self.collapse_features(conv)
        return conv